<h2>Lesson subject: avoid unnecessary computation with scan</h2>

We are using scan function to provided human-guided hint while searching for area that matches to particular address.

Let's use the same code to obtain data as in lesson a, but i na one cell.

In [ ]:
#load """Domain.fsx""" 
#load """Cities\Toronto.fsx""" 

open Domain

Toronto.license.info |> display 

let addresses , districts, streets, summary = 
    setup Toronto.data Area.Neighbourhoods (Limit.Streets 500) 

summary |> display

let positions = addresses |> Array.map (fun x -> x.Pos) |> Array.toList

let indexedDistricts = districts |> Array.indexed

Open Government Licence – Toronto
(https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-licence/)
Attribution: Contains information licensed under the Open Government Licence – Toronto

data summary (without limits) -> 
 areas: 140, streets: 9521, addressess: 527189 
 addresses per street (median/mean/std-dev): 32.0/55.3711795/104.1107272

Now we need to adapt our <i>hintSearch</i> human-driven algorithm slightly.
To mark computation as avoid or not we should  appropriate annotation. We could use true/false for that but let's use more descriptive type:

In [ ]:

let hintSearch lastHitAreaIndex currentPosition  =
    let hintArea = indexedDistricts.[lastHitAreaIndex]
    let find = snd >> isPointWithinFeature currentPosition

    if find hintArea then lastHitAreaIndex
    else indexedDistricts |> Array.findIndex find

let h::positions = positions
let init = hintSearch 0 h

let addressesAreas = (init, positions) ||> List.scan hintSearch 